# Using local installation of Ollama with Llama3.2 model to generate social media posts for events

**Pre-conditions:** Ollama should be installed on your local machine (for more information see the README.md file), so when you type
````ollama list````
it should show the list of installed models.
Run chat in the console with
````ollama run llama3.2````
 to download llama model for the first time.

If a package missing (import line gives ModuleNotFoundError), install it with pip (from a console) or from "Python Packages" window in the PyCharm.

Alternatively you can run the following code directly from this notebook, and it will install missing packages:

```
import sys
!"{sys.executable}" -m pip install jupyter ollama ipykernel
```

In [3]:
import ollama
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

***Specify the URL for the website below***

In [4]:
WEBSITE_URL = "https://www.eventbrite.ie/e/techstars-startup-week-guimaraes-tickets-1556700957469?aff=ebdssbdestsearch&keep_tld=1"
LANGUAGE = "English"
OLLAMA_API = "http://localhost:11434/api/chat"
MODEL = "llama3.2"

A class to represent a Webpage

In [5]:
# Some websites need you to use proper headers when fetching them:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}


class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

Create a system and user prompt

In [6]:
# Define system prompt and user prompt for summarization of a Website
def messages_summary(website, language):
    system_prompt = '''You are an assistant that analyzes the contents of a website \
        and provides a short summary, ignoring text that might be navigation related. \
        Respond in {0} only and format your response correctly in Markdown.'''.format(language)
    user_prompt = '''You are looking at a website titled {0}\n
        The contents of this website is as follows; \
        please provide a short summary of this website in markdown. \
        If it includes news or announcements, then summarize these too.\n\n"
        {1}'''.format(website.title, website.text)
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]


def messages_post(website_summary, language):
    system_prompt = '''You are an assistant that analyzes the contents of an event \
        and provides short summaries for a Twitter post, an Instagram post and a Facebook post.\
        Ensure the summaries are written in {0} and abide by the platform rules \
        for each of the platforms.'''.format(language)
    user_prompt = '''The events details are as follows: {0}. Please summarize the above.\
        Capture details like time and location, please capture them as well.'''.format(website_summary)
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]


# Generate a summary of content fetched by scraping the website
def summarize(url, language):
    website = Website(url)
    return ollama.chat(
        model=MODEL,
        messages=messages_summary(website, language)
    ).message.content


def write_post(website_summary, language):
    return ollama.chat(
        model=MODEL,
        messages=messages_post(website_summary, language)
    ).message.content

4## Main program
Depending on your machine might take 1-4 minutes to complete

In [7]:
summary = summarize(WEBSITE_URL, LANGUAGE)
post_text = str(write_post(summary, LANGUAGE)).strip()
display(Markdown(post_text))

Here are three social media summaries for the Techstars Startup Week Guimarães event:

**Twitter Post (280 characters)**
Join us in Guimarães, Portugal from Oct 13-17! Techstars Startup Week brings entrepreneurs, investors & innovators together to share ideas & grow startups. Tickets available on Eventbrite with a special offer: 50% off + buy 2 get 1 free! #TechstarsGuimarães

**Instagram Post**
Get ready to connect & innovate in Guimarães, Portugal! Techstars Startup Week is coming up from Oct 13-17 and promises an exciting experience for entrepreneurs, investors, and curious minds. Don't miss out on the opportunity to grow your startup! Tickets available now with a limited-time offer: 50% off + buy 2 get 1 free! Link in bio #TechstarsGuimarães

**Facebook Post**
Calling all startups & entrepreneurs! Techstars Startup Week Guimarães is just around the corner from Oct 13-17 and we want you to be there! This 3-day event will bring together like-minded individuals to share ideas, learn from each other, and grow your business. Plus, enjoy a special offer: 50% off + buy 2 get 1 free on tickets available now on Eventbrite! Don't miss out on this amazing opportunity! [Link to Eventbrite]

Note: I've kept the Facebook post longer as it's the platform with the longest character limit and allows for more detailed content. The Twitter and Instagram posts are shorter and more concise, while still conveying the essential information about the event.